# Read tweets stream from socket

In [ ]:
# import findspark
# import pyspark
# import sys
# import time
# findspark.init('/Users/abhisheksingh/Spark/spark-3.0.2-bin-hadoop2.7')

In [ ]:
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

sc = SparkContext("local[2]", "ReadTweetsFromSocket")
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream('localhost', 5555)
socket_lines = socket_stream.window(20)
print(type(socket_lines))

In [ ]:
from collections import namedtuple
fields = ('tag', 'count')
Tweet = namedtuple('Tweet', fields)

# Use paranthesis for multiple lines or use \
( socket_lines.flatMap( lambda text: text.split( " " ) ) #splits to a list
  .filter( lambda word: word.lower().startswith( "#" ) ) # checks for a hashtag call
  .map( lambda word: (word.lower(), 1)) # lower cases the word
  .reduceByKey( lambda a,b: a + b ) # reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # stores in a tweet object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc( "count") ) # sorts them in a DF
  .limit(10).registerTempTable('tweets') ) ) # registers to a table.


# Run twitterReadFromSocket at this point

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# below only works for Jupyter Notebooks!
%matplotlib inline

In [ ]:
ssc.start()

In [ ]:
count = 0
while count < 10:
    time.sleep(2)
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = (10, 8) )
    sns.barplot( x='count', y='tag', data=top_10_df)
    plt.show()
    count = count + 1

In [ ]:
ssc.stop()